In [25]:
"""
How does model works?
Machine learning bekerja dengan cara mengidentifikasi pola pada data yang ada untuk memprediksi target.
Esensi machine learning:
1. Memiliki pola
2. Belum memiliki formula untuk menentukan target, jika sudah ada formula untuk apa menggunakan machine learning.
3. Punya datanya
"""

# Basic data exploration
import pandas as pd

path = "../data/melb_data.csv"

df = pd.read_csv(path)

#hapus data kosong, 
# axis=0 artinya jika ada satu data yang kosong dalam satu baris maka semua data dalam baris tersebut akan dihapus
# axis=1 artinya jika ada satu data yang kosong dalam satu kolom maka semua data dalam kolom tersebut akan dihapus

df = df.dropna(axis=0)

#menghitung jumlah data
df.count()

Suburb           6196
Address          6196
Rooms            6196
Type             6196
Price            6196
Method           6196
SellerG          6196
Date             6196
Distance         6196
Postcode         6196
Bedroom2         6196
Bathroom         6196
Car              6196
Landsize         6196
BuildingArea     6196
YearBuilt        6196
CouncilArea      6196
Lattitude        6196
Longtitude       6196
Regionname       6196
Propertycount    6196
dtype: int64

In [33]:
#Import model
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(random_state=3)
#Menggunakan random_state agar hasilnya tidak berubah-ubah

#Mulai membuat model
"""
Untuk membuat meodel kita harus punya:
1. Feature (data yang digunakan untuk memprediksi) biasanya dilambangkan X
2. Target (data yang digunakan untuk hasil prediksi) biasanya dilambangkan y
"""

#Karena kita ingin memprediksi harga rumah, maka kita menggunakan 5 kolom untuk feature dan 1 kolom (Price) untuk target

#Features
df_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = df[df_features]
X.describe()

#Target
y = df.Price

#Latih data
model.fit(X,y)

#Hasil prediksi ML kita
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(model.predict(X.head()))

Making predictions for the following 5 houses:
   Rooms  Bathroom  Landsize  Lattitude  Longtitude
1      2       1.0     156.0   -37.8079    144.9934
2      3       2.0     134.0   -37.8093    144.9944
4      4       1.0     120.0   -37.8072    144.9941
6      3       2.0     245.0   -37.8024    144.9993
7      2       1.0     256.0   -37.8060    144.9954
The predictions are
[1035000. 1465000. 1600000. 1876000. 1636000.]


In [34]:
# Model Valdation
# Model Validation digunakan untuk mengukur seberapa tepat akurasi ML kita memprediksi hasil 
# Disini kita akan menggunakan tehnik MAE (Mean Absolute Error) untuk mengevaluasi model kita, semakin kecil MAE maka semakin baik Model kita
# MAE menghitung jumlah selisih antara y_test dengan y_pred dibagi jumlah data (n)
# y_test = data yang sudah kita sisipkan untuk test 
# y_pred = data yang kita prediksi

from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X)
mean_absolute_error(y, y_pred)


1115.7467183128902

In [45]:
# Memisahkan data untuk train dan test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 3)

# Jadi, kita akan menggunakan X_train dan y_train untuk fit model
# Lalu kita akan memprediksi menggunakan X_test
# hasil X_test itu akan kita ukur (evaluasi) dengan y_test(yang sudah pasti benar karena diambil langsung dari data pasangan X_test)

model_2 = DecisionTreeRegressor(random_state=3)
model_2.fit(X_train, y_train)

y_pred = model_2.predict(X_test)
mae = mean_absolute_error(y_test,y_pred)
mae

260995.34925758553

In [46]:
# Overfitting dan Underfitting 
"""
Overfitting yaitu ketika Model yang kita buat cocok dengan data pelatihan bahkan hampir sempurna, tapi ketika dicoba pada data baru model kita tidak
bekerja dengan baik. Overfitting ini terjadi karena model benar-benar memasukkan (menghapal) semua data dan tidak mencari polanya.
Underfitting kebalikannya, yaitu Model yang tidak bekerja dengan baik pada data train maupun data baru

Untuk mengatasi masalah Overfitting dan Underfitting maka kita harus memilih data yang bagus (tidak harus semua dimasukkan) untuk membuat model kita
"""

# Contoh dari kaggle
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)


for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

"""
Lihat outputnya, dari 5-500 leaf kita mendapatkan MAE yang semakin bagus (turun), tapi saat 5000 leaf MAE kembali naik.
Maka optimalnya ada pada 500 leaf
"""

Max leaf nodes: 5  		 Mean Absolute Error:  369937
Max leaf nodes: 50  		 Mean Absolute Error:  266677
Max leaf nodes: 500  		 Mean Absolute Error:  244752
Max leaf nodes: 5000  		 Mean Absolute Error:  262982


In [49]:
# Random Forests
# Dari Kaggle 
"""
Pohon yang dalam dengan banyak daun akan overfit karena setiap prediksi berasal dari data historis hanya dari beberapa rumah di daunnya. 
Tetapi pohon yang dangkal dengan sedikit daun akan berkinerja buruk karena gagal menangkap banyak perbedaan dalam data mentah.
"""
"""
Random Forests menggunakan banyak pohon, dan membuat prediksi dengan merata-ratakan prediksi setiap pohon komponen. 
Ini umumnya memiliki akurasi prediksi yang jauh lebih baik daripada DecisionTree dan bekerja dengan baik dengan parameter default. 
Jika Anda terus membuat model, Anda dapat mempelajari lebih banyak model dengan kinerja yang lebih baik, tetapi banyak di antaranya yang sensitif untuk mendapatkan parameter yang tepat.
"""

# Let's practice
from sklearn.ensemble import RandomForestRegressor
forest_model = RandomForestRegressor(random_state=3)
forest_model.fit(X_train, y_train)
forest_pred = forest_model.predict(X_test)
print(mean_absolute_error(y_test, forest_pred))

"""
Dengan menggunakan DecisionTree kita mendapatakan MAE 244rb sedangkan dengan menggunakan RandomForest kita mendapatkan MAE 188rb
Dalam kasus ini kita sedang memprediksi harga rumah, dapat kita lihat prediksi harga rumah dengan DecisionTree lebih mahal 56rb dibandingkan harga rumah dengan RandomForest
"""


188620.57381577467
